In [1]:
%load_ext autoreload
%autoreload 2

from experiments.all import train_and_evaluate
from tsm.configuration import TSMConfigParser
import docker
import logging

logging.basicConfig(level=logging.INFO)

# load configuration
config = TSMConfigParser()
config.read("/home/pablo/tesis/tsm-pipeline/code/configs/pablo-bestia.cfg")
config.check()
docker_client = docker.from_env()

In [2]:
from scripts.docker import read_dbs_dataset
from numpy.random import RandomState

sql_dbs = read_dbs_dataset("/home/pablo/tesis/tsm-pipeline/experiments/datasets/sql_1.txt")
# Using a fixed random state to have reproducible results
rand = RandomState(seed=5)

print(len(sql_dbs))

42
